In [2]:
import requests
import json
from config import api_key
import pandas as pd

In [ ]:
#Create a variable specifying electric and hybrid vehicles only
technology="hy,elec,phev,hev,nevs"
url=f"https://developer.nrel.gov/api/transportation-incentives-laws/v1.json?api_key={api_key}&technology={technology}"
data=requests.get(url).json()

In [76]:
#Out of the collected data
incentives_df=pd.DataFrame(data["result"])
incentives_df.head()

,id,state,title,text,enacted_date,amended_date,plaintext,is_recent,seq_num,type,agency,significant_update_date,recent_update_or_new,utility_id,county_ids,technologies,categories,types,references,topics
0,284,US,Congestion Mitigation and Air Quality (CMAQ) I...,<p>The CMAQ Program provides funding to state ...,2005-08-10T00:00:00Z,2021-11-15T00:00:00Z,The CMAQ Program provides funding to state dep...,False,54.0,Incentives,U.S. Department of Transportation,11/29/2021,update,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[BIOD, ETH, ELEC, HY, IR, NG, PHEV, LPG]","[{'code': 'STATION', 'title': 'Alternative Fue...","[{'id': 8, 'title': 'Incentives', 'code': 'INC'}]","[{'description': 'Public Law 117-58', 'url': '...",[]
1,288,US,Clean Cities Coalition Network,The mission of Clean Cities Coalition Network ...,None,None,The mission of Clean Cities Coalition Network ...,False,21.0,Programs,U.S. Department of Energy,None,update,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[AFTMKTCONV, BIOD, ETH, ELEC, EFFEC, HEV, HY, ...","[{'code': 'AFTMKTCONV', 'title': 'Aftermarket ...","[{'id': 9, 'title': 'Programs', 'code': 'PROG'}]",[],[]
2,317,US,State Energy Program (SEP) Funding,The SEP provides grants to states to assist in...,None,2021-11-15T00:00:00Z,The SEP provides grants to states to assist in...,False,26.0,Incentives,U.S. Department of Energy,11/29/2021,update,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[BIOD, ETH, ELEC, EFFEC, HY, NG, PHEV, LPG]","[{'code': 'BIOD', 'title': 'Biodiesel', 'categ...","[{'id': 8, 'title': 'Incentives', 'code': 'INC'}]","[{'description': 'Public Law 117-58', 'url': '...",[]
3,319,US,Alternative Fuel Excise Tax Credit,<p>NOTE: This incentive was originally set to ...,2005-08-10T00:00:00Z,2022-08-16T00:00:00Z,NOTE: This incentive was originally set to exp...,False,11.0,Incentives,U.S. Internal Revenue Service,8/24/2022,update,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[HY, NG, OTHER, LPG]","[{'code': 'STATION', 'title': 'Alternative Fue...","[{'id': 8, 'title': 'Incentives', 'code': 'INC'}]","[{'description': '26 U.S. Code 6426', 'url': '...",[]
4,323,US,Clean School Bus,<p>The U.S. Environmental Protection Agency’s ...,None,2021-11-15T00:00:00Z,The U.S. Environmental Protection Agency's (EP...,False,33.0,Incentives,U.S. Environmental Protection Agency,8/16/2022,update,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[AFTMKTCONV, ETH, ELEC, EFFEC, HY, IR, NG, PHE...","[{'code': 'AFTMKTCONV', 'title': 'Aftermarket ...","[{'id': 8, 'title': 'Incentives', 'code': 'INC'}]","[{'description': 'Public Law 117-58', 'url': '...",[]


In [77]:
#We are interested in having an amount of all different types of incentives and regulations per state.
reduced_df=incentives_df[["id","state","type"]]
#drop empty values "type" values:
reduced_df=reduced_df.loc[reduced_df["type"]!=""].copy()
reduced_df

,id,state,type
0,284,US,Incentives
1,288,US,Programs
2,317,US,Incentives
3,319,US,Incentives
4,323,US,Incentives
...,...,...,...
1305,13151,CA,Laws and Regulations
1306,13152,CA,Utility/Private Incentives
1307,13157,IL,Laws and Regulations
1308,13158,CO,State Incentives


In [87]:
incentive_types_df=pd.DataFrame(index=reduced_df["state"].unique())
for type in reduced_df["type"].unique():
    incentive_types_df[type]=reduced_df.loc[reduced_df["type"]==type].groupby("state").count()["id"]
incentive_types_df.head()

,Incentives,Programs,Laws and Regulations,State Incentives,Utility/Private Incentives
US,37.0,4.0,20,NaN,NaN
AZ,NaN,NaN,13,6.0,13.0
CA,NaN,NaN,61,41.0,52.0
CO,NaN,NaN,25,13.0,14.0
DC,NaN,NaN,13,4.0,2.0


In [86]:
incentive_types_df.to_csv("Resources/incentives.csv")